In [180]:
import camelot
import pandas as pd
import numpy as np
import pdfminer as pm
import math
from datetime import datetime  
from datetime import timedelta  
import warnings
import os
import subprocess
import ocrmypdf
import sys
import traceback

In [7]:
print(len(p1t))

0


In [103]:
## Read in all table regions
def convertPDFToTable(filepath):
    table_cols = [['35,52,97,136,210,256,308,360,390,422,456,469,490,513,550',
                  '35,52,97,136,210,256,308,360,390,422,456,469,490,513,550',
                  '35,52,97,136,210,256,308,360,390,422,456,469,490,513,550',
                  '35,52,97,136,210,256,308,360,390,422,456,469,490,513,550',
                  '35,52,97,136,210,256,308,360,390,422,456,469,490,513,550'], 
                  ['44,']]
    curColSetting = 0
    p1t = camelot.read_pdf(filepath, 
                         strip_text=' .\n', flavor='stream', pages="1-end",
                         #table_regions=['0,700,600,0'],
                          columns=table_cols[curColSetting], 
                           edge_tol=100)
    pdf = []
    for t in p1t:
        pdf.append(t.df)
    return pdf

In [157]:
def processOrder(tables, committee, station):
    progColre=False
    ## Create the ad day/times df.
    ads = pd.DataFrame(columns=['StartDate', 'EndDate', 'Weekdays', 'Spots/Week', 'Rate', 'Rating', 'index'])
    ## Create the shows DF.
    shows = pd.DataFrame(columns=['index', 'Title'])
    secondlines = pd.DataFrame(columns=['index', 'Title'])
    ## Add second line of inventory code
    def mergeSLines(row):
        if row.name+1 < len(shows):
            nextRow = shows.iloc[row.name+1]
            sline = secondlines[(row['index'] < secondlines['index']) & (nextRow['index']>secondlines['index'])]
        else: 
            sline = secondlines[(row['index'] < secondlines['index'])]
        if(len(sline)>0):
            if(row['Title'] != sline['Title'].values[0]):
                row['Title'] = row['Title'] + sline['Title'].values[0]
        else:
            print('Second line merge error at ad: ', row.name)
        return row
    
    def getSecondLines(df):
        returnVal = pd.DataFrame(columns=['index', 'Title'])
        #Get show lines
        progs = df[(df['Amount'] != '') & (df['Ch'] != '')]
        progs = progs.iloc[1:] # Remove residual header row.
        slines = list(map(lambda x: x+1, progs.index.values))
        if not (slines[-1] < len(df)): ## In case the second show line is on the next page.
            print('Program second line on next page.')
            slines = slines[:-1]
        slines = df.iloc[slines]
        slines = slines.replace('', np.nan, regex=True)
        slines = slines[slines.isnull().sum(axis=1) >= len(slines.columns)-2]
        for index, row in slines.iterrows():
            nulls = pd.isnull(row)
            nulls = nulls[nulls!=True]
            returnVal = returnVal.append({'index': index, 'Title': row[nulls.index[0]]}, ignore_index=True)
        returnVal['index'] = returnVal['index'] + lastIndex
        return returnVal
    def addNameIfDiff(p):
        for val in nlines.iloc[p.name]:
            if(p[progColName] != val) & (val.isna() != True):
                p[progColName] = p[progColName] + val
        return p
    ## Merge program names into ad days.
    def mergeProgs(i):
        p = shows[shows['index']<=i]
        if len(p)>0:
            p = p.iloc[len(p)-1]
            return p['Title']
        else:
            return ""
    ## Create individual records for each ad spot.  
    ## Filter out day times rows.                
    def getDays(row):
        #print(row)
        nextRow = 1
        dayCols = ['StartDate', 'EndDate', 'Weekdays', 'Spots/Week', 'Rate']
        for val in row:
            try:
                ind = dayCols.index(val)
                dayCols.remove(val)
            except:
                pass
        if(len(dayCols) == 0):
            if(len(headers)==0):
                headers.append(buys.loc[(row.name), :])
            while (nextRow!=-1):
                dayColFound=False
                if((row.name+nextRow) != len(buys)):
                    potRow = buys.loc[(row.name+nextRow), :]
                    valCount=0
                    for k in potRow:
                        if 'Week:' in k:
                            if len(k.split(':'))>1:
                                fixVal = k.split(':')[1]
                                potRow[valCount+1] = fixVal
                            nextRow=nextRow+1
                            dayColFound=True
                        valCount = valCount+1
                    if dayColFound != True:
                        nextRow=-1
                    else:
                        headers.append(potRow)
                else:
                    nextRow=-1
    def findHeaderRow(row):
        headers =  { 'Description': ['StartDateEndDateDescription'],
                     'InventoryCode': ['Amount', 'Start', 'InventoryCode', 'Rate', 'Spots'] }
        for htype in headers:
            for val in row:
                try:
                    ind = headers[htype].index(val)
                    headers[htype].remove(val)
                except:
                    pass
        if(len(headers['Description']) == 0):
            return ['Description', row.name]
        else:
            if(len(headers['InventoryCode']) == 0):
                return ['InventoryCode', row.name]
    def fixHeaders(row):
        fixCol = -1
        for k in row.keys():
            if row[k] == 'StartDateEndDateDescription':
                fixCol = k
        if fixCol != -1:
            if(fixCol == 3):
                row[(fixCol-2)] = 'Ch'
                row[(fixCol-1)] = 'Start'
                row[(fixCol)] = 'End'
                row[(fixCol+1)] = 'Description'
                row[(fixCol+2)] = 'Desc2'
            if(fixCol == 2):
                row[(fixCol)] = 'Start'
                row[(fixCol+1)] = 'End'
                row[(fixCol+2)] = 'Description'
        return row
    ## Filter out program names.
    lastIndex = 0
    for t in tables:
        progColName=False
        headerConfig = t.apply(findHeaderRow, axis=1).dropna()
        if len(headerConfig)>0:
            progColName = headerConfig.values[0][0]
        if progColName != False:
            ## Figure out inventory code lines
            progs = t.copy()
            if progColName == 'Description':
                progs = progs.apply(fixHeaders, axis=1)
                progs.columns = progs.iloc[headerConfig.values[0][1]]
            else:
                progs.columns = progs.iloc[headerConfig.values[0][1]]
            progs = progs.iloc[headerConfig.values[0][1]:]
            progs = progs.reset_index(drop=True)
            
            ## Some WideOrbit reports have two line program names, this gets the second lines.
            if progColName == 'InventoryCode':
                slines = getSecondLines(progs)
                secondlines = secondlines.append(slines)
            ## Get the program name rows
            progs = progs[(progs['Amount'] != '') & (progs['Ch'] != '')] 
            progs = progs.iloc[1:] # Remove residual header row.
            if progColName == 'Description':
                progs['Description'] = progs['Description'] + progs['Desc2']
            progs = progs.reset_index()
            ## Make index run continously accross pages.
            progs['index'] = progs['index'] + lastIndex
            lastIndex = t.iloc[len(t)-1].name+lastIndex+1
            ## Remove duplicate columns and append to the master list.
            progs = progs.loc[:,~progs.columns.duplicated()]
            progs = progs.rename(columns={progColName:'Title'})
            progs = progs[['index', 'Title']]
            shows = shows.append(progs, sort=False)
        else:
            pass
            #print("False")
    ## Merge in the second lines to complete the program names.
    if progColName == 'InventoryCode':
        shows = shows.reset_index(drop=True)
        shows = shows.apply(mergeSLines, axis=1)
    ## Seperate out the days and times each ad will be on.
    lastIndex = 0
    for t in tables:
        progColName=False
        headerConfig = t.apply(findHeaderRow, axis=1).dropna()
        if len(headerConfig)>0:
            progColName = headerConfig.values[0][0]
        if progColName != False:
            headers = []
            buys = t.copy()
            buys = buys.dropna(how='all')
            buys = buys.iloc[headerConfig.values[0][1]:]
            buys = buys.reset_index(drop=True)
            buys.apply(getDays, axis=1)
            days = pd.DataFrame(headers)
            days = days.replace('', np.nan)
            days = days.dropna(axis='columns')
            days.columns = days.iloc[0]
            days = days[1:]
            days = days.reset_index()
            days['index'] = days['index'] + lastIndex
            lastIndex = t.iloc[len(t)-1].name+lastIndex+1
            ads = ads.append(days, sort=False)
    ## Add in Program Names
    ads['Program'] = ads['index'].apply(mergeProgs)
    ads = ads.drop(columns=['index'], axis=1)
    '''
    print('SHOWS')
    print(shows)
    print('ADS')
    print(ads[['index', 'Weekdays']])
    print(len(adtimes))
    '''
    ## Expand each ad buy listing to individual spots
    return ads

In [198]:
def parseCyclePDFs(cyc, basepath, test):
    fulladlist = pd.DataFrame(columns=['Date', 'Rate', 'Committee', 'Station', 'Program'])
    pdfResults = pd.DataFrame(columns=['Path', 'Result'])
    kw = pd.read_csv('/media/andrew/F08C9B848C9B444E/analysis/tv/fccscraper/keys/filetypekeywords.csv') # keywords
    malads = pd.DataFrame(columns=['StartDate', 'EndDate', 'Weekdays',
                                   'Spots/Week', 'Rate', 'Rating'])
    def digToNextLevel(folderpath, name):
        nonlocal pdfResults
        nonlocal fulladlist
        nonlocal kw
        adtimes = [] # Final add array
        stationads = pd.DataFrame(columns=['StartDate', 'EndDate', 'Weekdays',
                                           'Spots/Week', 'Rate', 'Rating'])
        def expandDays(x):
            adStartDate = datetime.strptime(x['StartDate'], '%m%d%y')
            # Test if adbuy string is malformed.
            for i, day in enumerate(x['Weekdays']):
                if(day!='-'):
                    if(day.isdigit()):
                        for j in range(0, int(day)):
                            adtimes.append({
                                        'Date': (adStartDate + timedelta(days=i)),  
                                        'Rate': x['Rate'],
                                        'Committee': name,
                                        'Station': station,
                                        'Program': x['Program']
                                       })
                    else: ## CODE FOR MTW NOTATION
                        adtimes.append({
                                        'Date': (adStartDate + timedelta(days=i)),  
                                        'Rate': x['Rate'],
                                        'Committee': name,
                                        'Station': station,
                                        'Program': x['Program']
                                       })
        def classifyReportFormat(res, kw):
            # Process the scraped text
            res = res.split('\n')
            res = [x.replace(' ', '') for x in res ]
            res = [x.replace(':', '') for x in res ]
            res = [x.replace('.', '') for x in res ]
            res = list(filter(lambda a: a != '', res))
            res = res[0:200]
            res = list(set(res))
            # Generate keys from csv and score based on keys
            keyAr = kw['pdftype'].values
            ptypes = {}
            for pt in keyAr:
                ptkw = kw[kw['pdftype']==pt]
                ptypes[pt] = len(ptkw[ptkw['keyword'].isin(res)])
                ptypes[pt] = ptypes[pt]/len(ptkw)
            maxVal = max(ptypes, key=ptypes.get)
            if ptypes[maxVal] > .2:
                return maxVal
            else:
                return False
        def checkIntegrity(ad): # Checks ads are in valid format, performs common corrections due to OCR errors.
            nonlocal stationads
            malformed=False
            # Date
            for d in ['StartDate', 'EndDate']:
                ad[d] = ad[d].replace('/', '')
                ad[d] = ad[d].replace('o', '0')
                ad[d] = ad[d].replace('O', '0')
                ad[d] = ad[d].replace('g', '9')
                try:
                    datetime.strptime(ad[d], '%m%d%y')
                except:
                    malformed=True            
            # Rate
            ad['Rate'] = ad['Rate'].split('$')[1]
            ad['Rate'] = ad['Rate'].replace(',','')
            try:
                ad['Rate'] = float(ad['Rate'])
                ad['Rate'] = ad['Rate']/100
            except:
                traceback.print_exc()
                malformed=True
            # Weekday
            validDateChars=['M', 'T', 'W', 'h', 'F', 'S', 'a', 'u', 'H', 'A', 'U']
            if len(ad['Weekdays']) != 7:
                malformed=True
            for char in ad['Weekdays']:
                if char.isalpha():
                    if char not in validDateChars:
                        malformed=True
            if malformed!=True:
                stationads = stationads.append(ad, ignore_index=True, sort=False)
            # Maybe malformed sheet else?
        def processPDF(item, station, name):
            nonlocal pdfResults
            nonlocal fulladlist
            nonlocal stationads
            pdfProcessors = {'orders': processOrder, 'contracts': processOrder}
            result = subprocess.run(['pdftotext', item.path, '-'], 
                                        stdout=subprocess.PIPE).stdout.decode()

            if((len(result.split('\n')[0:30])> 5)):
                reportFormat = classifyReportFormat(result, kw)
                if (reportFormat != False) & (reportFormat in pdfProcessors != False) :
                    print(item.path)
                    pdfTable = convertPDFToTable(item.path)
                    try:
                        ads = pdfProcessors[reportFormat](pdfTable, name, station)
                        display(ads)
                        if len(ads) > 0:
                            ads.apply(checkIntegrity, axis=1)
                            pdfResults = pdfResults.append({'Path': item.path, 
                                                            'Result': reportFormat+' Success'}, ignore_index=True, sort=False)
                        else:
                            pdfResults = pdfResults.append({'Path': item.path, 
                                            'Result': reportFormat+' Scraping Error'}, ignore_index=True)
                    except:
                        traceback.print_exc()
                        pdfResults = pdfResults.append({'Path': item.path, 
                                            'Result': 'Scraping Error'}, ignore_index=True)
                else:
                    if reportFormat != False:
                        pdfResults = pdfResults.append({'Path': item.path, 
                           'Result': reportFormat+'No parser written for file.'}, ignore_index=True)
                    else:
                        pdfResults = pdfResults.append({'Path': item.path, 
                           'Result': 'No parser written for file.'}, ignore_index=True)
            else:
                print('OCR')
                ocrmypdf.ocr(item.path, item.path, deskew=True, rotate_pages=True)
                processPDF(item, station, name)
        for item in list(os.scandir(folderpath)):
            if (item.is_file()):
                processPDF(item, station, name)
            else:
                digToNextLevel(item.path, item.name)
        if len(stationads)>0:
            stationads = stationads.drop_duplicates() # subset array arg to ignore columns
            stationads.apply(expandDays, axis=1)
            fulladlist = fulladlist.append(pd.DataFrame(adtimes), ignore_index=True)
    if test:
        station = 'test'
        digToNextLevel(basepath, cyc)
    else:
        for station in list(os.scandir(basepath)):
            station = station.name
            cyclepath = basepath+station+'/Political Files/'+cyc+'/'
            digToNextLevel(cyclepath, cyc)
        fulladlist['Cycle'] = cyc
    return {'ads': fulladlist, 'pdfs': pdfResults, 'malformedads': malads}
results = parseCyclePDFs('2018', '/media/andrew/F08C9B848C9B444E/analysis/tv/ocrtest', test=True)

/media/andrew/F08C9B848C9B444E/analysis/tv/ocrtest/KMSB Order 1592727 AZ EDUCATION PROJECT 4.28-4.29.18.pdf


,StartDate,EndDate,Weekdays,Spots/Week,Rate,Rating,Program
0,04/23/18,04/29/18,------1,1,$4500,000,
1,04/23/18,04/29/18,------1,1,$22500,000,LocalNews@9pSu9:00PM-10:00PM
2,04/23/18,04/29/18,-----1-,1,$17500,000,LocalNews@9pSa


/media/andrew/F08C9B848C9B444E/analysis/tv/ocrtest/KTTU Order 1652036 SINEMA FOR SENATE 9.6-9.12.18.pdf


,StartDate,EndDate,Weekdays,Spots/Week,Rate,Rating,Program
0,09/03/18,09/09/18,---TF--,2,$1000,000,
1,09/10/18,09/16/18,M-W----,2,$1000,"0,00",1-2pM-FM-F1p-2p
2,09/10/18,09/16/18,===,1,$2000,000,2-3pM-FM-F2p-3p
3,09/03/18,09/09/18,---TF--,2,$15000,000,6-630pM-FM-F6p-630p
4,09/03/18,09/09/18,~~TF--,2,$15000,000,630-7pM-F6:30PM-7:00PM
5,09/10/18,09/16/18,MTW---—,3,$15000,000,630-7pM-FM-F630p-7p
6,09/03/18,09/09/18,-----1-,1,$10000,000,6-630pSa6:00PM-6:30PM
7,09/03/18,og/09/18,-----1-,1,$10000,000,630-7pSaSa630p-7p
8,09/03/18,09/09/18,------1,1,$10000,000,6-630pSuSu6p-630p
9,09/03/18,09/09/18,------1,1,$10000,000,630-7pSuSu630p-7p


/media/andrew/F08C9B848C9B444E/analysis/tv/ocrtest/KTTU Order 1675477 SINEMA FOR SENATE 10.12-10.15.18.pdf


,StartDate,EndDate,Weekdays,Spots/Week,Rate,Rating,Program
0,10/08/18,10/14/18,----1--,NaN,$65000,000,
1,10/15/18,10/21/18,teseeee,NaN,1$65000,000,6-630pM-FM-F6p-630p
2,10/08/18,10/14/18,----1--,NaN,$65000,000,630-7pM-FM-F630p-7p
3,10/08/18,10/14/18,-----1-,NaN,$40000,000,6-630pSa6:00PM-6:30PM
4,10/08/18,10/1418,-----1-,NaN,$40000,000,630-7pSaSa630p-7p
5,10/08/18,10/1418,------1,NaN,$40000,000,6-630pSuSu6p-630p
6,10/08/18,10/1418,------1,NaN,$40000,000,630-7pSuSu630p-7p
7,10/15/18,10/21/18,]------,NaN,$20000,000,MonPrimeB


In [160]:
results['pdfs']['Path'] = results['pdfs']['Path'].str[-40:]
results['pdfs']

,Path,Result
0,27 AZ EDUCATION PROJECT 4.28-4.29.18.pdf,orders Success
1,652036 SINEMA FOR SENATE 9.6-9.12.18.pdf,orders Success
2,477 SINEMA FOR SENATE 10.12-10.15.18.pdf,orders Success


In [197]:
results['ads'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 5 columns):
Date         48 non-null datetime64[ns]
Rate         48 non-null float64
Committee    48 non-null object
Station      48 non-null object
Program      48 non-null object
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 2.0+ KB


In [48]:
results['ads'][results['ads']['Program'].str.contains('Prime')]

,Date,Rate,Committee,Station,Program


In [6]:
ocrmypdf.ocr('input.pdf', 'output.pdf', deskew=True, rotate_pages=True)

62393

In [43]:
# Generating keys for predicting what form a PDF is in.
keys = {}
for ftype in list(os.scandir('/media/andrew/F08C9B848C9B444E/analysis/tv/orderscoring/')):
    o = list(os.scandir('/media/andrew/F08C9B848C9B444E/analysis/tv/orderscoring/'+ftype.name+'/'))
    op = []
    # Convert all pdfs to text, string process them and turn them into an array of strings
    for f in o:
        res = subprocess.run(['pdftotext', f.path, '-'], 
                                            stdout=subprocess.PIPE).stdout.decode()
        res = res.split('\n')
        res = [x.replace(' ', '') for x in res ]
        res = [x.replace(':', '') for x in res ]
        res = [x.replace('.', '') for x in res ]
        res = list(filter(lambda a: a != '', res))
        res = res[0:200]
        res = list(set(res))
        #print(res[0:100])
        if len(res)>1:
            op.append(res)
        else:
            ocrmypdf.ocr(f.path, f.path, deskew=True, rotate_pages=True)
    curSet = []
    # Filter so only keys that exist in all files of the given report format remain.
    print(len(op))
    for l in op:
        if len(curSet)==0:
            curSet=l
        else:
            curSet = [x for x in curSet if x in l]
    keys[ftype.name] = curSet
# Filter out keys that are non-unique to that report type.
for k in keys:
    types = ['contracts', 'invoices', 'orders']
    types = [x for x in types if x != k]
    for t in types:
        keys[k] = [x for x in keys[k] if x not in keys[t]]
    # Get rid of nonspecific keys
    keys[k] = list(filter(lambda a: (len(a)>4) & (len(a)<23) , keys[k]))
    print(keys[k])
keys = [[(k, vv) for vv in v] for k, v in keys.items()]
keys2 = []
for k in keys:
    for v in k:
        keys2.append(v)
keys = pd.DataFrame(keys2, columns=['pdftype', 'keyword'])
keys.to_csv('filetypekeywords.csv', index=False)
#print(curSet)
#print(1)


6
9
6
['Weekdays', 'StartDateEndDate', 'LengthWeek', 'Spots/', 'OriginalDate/Revision', 'PrintDate', 'Product1/2', 'Amount', 'AdvertiserCode', 'Start/End', 'Cash/Trade', 'Contract/Revision', 'Spots/Week']
['BillingAddress', 'Deal#', 'Invoice#', 'AgencyRef', 'Order#', 'Property', 'Product', 'AdvertiserRef']
['#Spots', 'OriginalDate/Rev', 'Estimate', 'Demographic', 'OrderSeparation', 'ORDER', 'PrimaryAE', 'BillPlan', 'Priority', 'UnitCode', 'Agency', 'GrossAmount', 'BillingCalendar', 'ProductCodes', 'Advertiser', 'FlightDates', 'Order%', 'Order/Rev', 'StartDate/EndDate', 'OrderType', 'AdvertiserExternalID', 'AccountExecutives', 'Month', 'BillingCycle', 'EOM/EOC', 'InventoryCode', 'BillingType', 'Start', 'Orders', 'AltOrder#', 'AgencyExternalID', 'RevenueCode2', 'AgencyCommission', 'Totals', 'SalesOffice', 'RevenueCode3', 'StartDate', 'SalesRegion', 'NewBusinessThru', 'BillingContact', 'NetAmount', 'AccountExecutive', 'RevenueCode1', '#SpotsGrossAmount', 'BuyingContact', 'ProductDesc', 'E

In [173]:
len('—')

1